## How to get Matrix P from camera model parameters

In [1]:
import numpy as np
import math

In [2]:
def rodrigues(r):
    '''Rodrigues formula
    Input: 1x3 array of rotations about x, y, and z
    Output: 3x3 rotation matrix'''
    def S(n):
        Sn = np.array([
            [0, -n[2], n[1]],
            [n[2], 0, -n[0]],
            [-n[1], n[0], 0]])
        return Sn
    theta = np.linalg.norm(r)
    if theta > 1e-30:
        n = r/theta
        Sn = S(n)
        R = np.eye(3) + np.sin(theta)*Sn + (1-np.cos(theta))*np.dot(Sn, Sn)
    else:
        Sr = S(r)
        theta2 = theta**2
        R = np.eye(3) + (1 - theta2 / 6.)*Sr + (.5 - theta2 / 24.) * np.dot(Sr, Sr)
    return np.mat(R)

In [ ]:
def matrixP_from_camera_model(height, width, zoom, skew, pan, tilt, roll, Tx, Ty, Tz):
    """
    Creates projection matrix P from camera model parameters
    :param height: camera image height
    :param width: camera image width
    :param zoom: camera focal
    :param skew:
    :param pan:
    :param tilt:
    :param roll:
    :param Tx:
    :param Ty:
    :param Tz:
    :return: the projection Matrix P
    
    """
    aspect_ratio = float(width)/float(height)
    K = np.array([[Zoom, Skew, 0.5], [0.0, Zoom * AspectRatio, 0.5], [0.0, 0.0, 1.0]])

    # Rotation matrix
    Rpan = rodrigues(np.array([0.0, 0.0, pan*math.pi/180.0]))
    Rtilt = rodrigues(np.array([tilt*math.pi/180.0, 0.0, 0.0]))
    Rroll = rodrigues(np.array([0.0, 0.0, roll*math.pi/180.0]))
    Mrot = Rroll * Rtilt * Rpan

    # Translation vector
    t = np.array([Tx, Ty, Tz])
    KR = K * Mrot
    Kt = np.dot(K, t)

    # Projection Martix
    Rproj = np.zeros((3, 4))
    Rproj[:, 0] = KR[:, 0].T
    Rproj[:, 1] = KR[:, 1].T
    Rproj[:, 2] = KR[:, 2].T
    Rproj[:, 3] = Kt

